<a href="https://colab.research.google.com/github/Francisss3/Data-Engineering/blob/main/Complex_queries_Mongodb_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pymongo[srv] dnspython certifi


In [2]:
import certifi
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

# Atlas credentials
username = "charbelfrancis03"
password = "hUmsVOJeW3LRlLZ6"

# build the SRV URI
uri = (
    f"mongodb+srv://{username}:{password}"
    "@cluster0.8otd0br.mongodb.net"
    "/?retryWrites=true&w=majority"
)

# create the client using certifi’s CA bundle
client = MongoClient(
    uri,
    server_api=ServerApi("1"),
    tls=True,
    tlsCAFile=certifi.where()
)

# test the connection
try:
    client.admin.command("ping")
    print("✅ Connected to MongoDB Atlas!")
except Exception as e:
    print("❌ Connection failed:", e)


✅ Connected to MongoDB Atlas!


In [7]:
# List all collections in the “ecommerce” database
print("Collections:", db.list_collection_names())

# If there is an 'orders' collection, count how many docs are in it
count = orders.count_documents({})
print("orders.count =", count)

# Print up to 5 sample documents (if any exist)
print("Sample orders:")
for doc in orders.find().limit(5):
    print(doc)


Collections: ['products']
orders.count = 0
Sample orders:


In [8]:
import datetime

# 1) Define a few sample orders
sample_orders = [
    {
        "order_id": 1001,
        "customer": {"name": "John Doe",  "email": "john@example.com"},
        "items": [
            {"product": "Laptop",     "quantity": 1, "price": 1200},
            {"product": "Headphones", "quantity": 2, "price": 150}
        ],
        "total": 1500,
        "order_date": datetime.datetime(2025, 6, 16, 10, 0)
    },
    {
        "order_id": 1002,
        "customer": {"name": "Jane Smith","email": "jane@example.com"},
        "items": [
            {"product": "T-Shirt", "quantity": 3, "price": 20}
        ],
        "total": 60,
        "order_date": datetime.datetime(2025, 6, 15, 14, 30)
    },
    {
        "order_id": 1003,
        "customer": {"name": "John Doe",  "email": "john@example.com"},
        "items": [
            {"product": "Sofa",  "quantity": 1, "price": 700}
        ],
        "total": 700,
        "order_date": datetime.datetime(2025, 6, 14, 9, 10)
    }
]

# 2) Insert into Atlas collection
result = orders.insert_many(sample_orders)
print("Inserted order_ids:", result.inserted_ids)

# 3) Verify
print("New count:", orders.count_documents({}))


Inserted order_ids: [ObjectId('6850192c29dad0e28659d318'), ObjectId('6850192c29dad0e28659d319'), ObjectId('6850192c29dad0e28659d31a')]
New count: 3


In [9]:
print("Collections:", db.list_collection_names())
print("orders.count =", orders.count_documents({}))
for doc in orders.find().limit(5):
    print(doc)


Collections: ['orders', 'products']
orders.count = 3
{'_id': ObjectId('6850192c29dad0e28659d318'), 'order_id': 1001, 'customer': {'name': 'John Doe', 'email': 'john@example.com'}, 'items': [{'product': 'Laptop', 'quantity': 1, 'price': 1200}, {'product': 'Headphones', 'quantity': 2, 'price': 150}], 'total': 1500, 'order_date': datetime.datetime(2025, 6, 16, 10, 0)}
{'_id': ObjectId('6850192c29dad0e28659d319'), 'order_id': 1002, 'customer': {'name': 'Jane Smith', 'email': 'jane@example.com'}, 'items': [{'product': 'T-Shirt', 'quantity': 3, 'price': 20}], 'total': 60, 'order_date': datetime.datetime(2025, 6, 15, 14, 30)}
{'_id': ObjectId('6850192c29dad0e28659d31a'), 'order_id': 1003, 'customer': {'name': 'John Doe', 'email': 'john@example.com'}, 'items': [{'product': 'Sofa', 'quantity': 1, 'price': 700}], 'total': 700, 'order_date': datetime.datetime(2025, 6, 14, 9, 10)}


In [10]:
# Task 1 – Filtering Data
# Find all orders by “John Doe” with total > 500,
# and project only order_id & total.

cursor = orders.find(
    {
        "customer.name": "John Doe",   # nested field match
        "total": {"$gt": 500}          # total greater than $500
    },
    {
        "_id": 0,                      # hide the Mongo _id
        "order_id": 1,
        "total": 1
    }
)

print("Orders by John Doe over $500:")
for doc in cursor:
    print(doc)


Orders by John Doe over $500:
{'order_id': 1001, 'total': 1500}
{'order_id': 1003, 'total': 700}


In [11]:
# Task 2 – Sorting Data
# Retrieve all orders, sorted by:
#  1) order_date descending  (newest first)
#  2) total ascending         (cheapest first among same date)

results = orders.find(
    {},
    {"_id": 0, "order_id": 1, "order_date": 1, "total": 1}
).sort([
    ("order_date", -1),  # -1 = descending
    ("total", 1)         #  1 = ascending
])

print("All orders, sorted by date ↓ and total ↑:")
for doc in results:
    print(doc)


All orders, sorted by date ↓ and total ↑:
{'order_id': 1001, 'total': 1500, 'order_date': datetime.datetime(2025, 6, 16, 10, 0)}
{'order_id': 1002, 'total': 60, 'order_date': datetime.datetime(2025, 6, 15, 14, 30)}
{'order_id': 1003, 'total': 700, 'order_date': datetime.datetime(2025, 6, 14, 9, 10)}


In [12]:
# Task 3 – Aggregation: Total Sales per Product
# 1) unwind the items array so each item is its own document
# 2) group by items.product
# 3) sum quantity * price to get revenue per product

pipeline = [
    {"$unwind": "$items"},
    {
        "$group": {
            "_id": "$items.product",
            "totalSales": {
                "$sum": { "$multiply": ["$items.quantity", "$items.price"] }
            }
        }
    }
]

agg_result = list(orders.aggregate(pipeline))
print("Total revenue per product:")
for r in agg_result:
    print(r)


Total revenue per product:
{'_id': 'Laptop', 'totalSales': 1200}
{'_id': 'Headphones', 'totalSales': 300}
{'_id': 'Sofa', 'totalSales': 700}
{'_id': 'T-Shirt', 'totalSales': 60}


In [13]:
# Task 4 – Aggregation: Average Order Value per Customer
# 1) group by customer.name
# 2) average the 'total' field

pipeline = [
    {
        "$group": {
            "_id": "$customer.name",
            "averageOrderValue": { "$avg": "$total" }
        }
    }
]

avg_result = list(orders.aggregate(pipeline))
print("Average order value per customer:")
for r in avg_result:
    print(r)


Average order value per customer:
{'_id': 'John Doe', 'averageOrderValue': 1100.0}
{'_id': 'Jane Smith', 'averageOrderValue': 60.0}


In [14]:
# Task 5 – Top 5 Products by Quantity Sold
# 1) unwind items
# 2) group by product, summing quantity
# 3) sort descending by quantitySold
# 4) limit to top 5

pipeline = [
    {"$unwind": "$items"},
    {
        "$group": {
            "_id": "$items.product",
            "quantitySold": { "$sum": "$items.quantity" }
        }
    },
    {"$sort": {"quantitySold": -1}},
    {"$limit": 5}
]

top5 = list(orders.aggregate(pipeline))
print("Top 5 products by quantity sold:")
for r in top5:
    print(r)


Top 5 products by quantity sold:
{'_id': 'T-Shirt', 'quantitySold': 3}
{'_id': 'Headphones', 'quantitySold': 2}
{'_id': 'Sofa', 'quantitySold': 1}
{'_id': 'Laptop', 'quantitySold': 1}
